https://stepik.org/course/116332

https://github.com/glebmikha

https://github.com/glebmikha/sql-course
    
https://colab.research.google.com/drive/1Og4wDz-BELxR6izJyWFX-Wn3HVFPHE3W?usp=sharing

In [1]:
import sqlite3
import pandas as pd

In [2]:
sqlite3.sqlite_version

'3.33.0'

In [3]:
# # connect base 'db' (create/write)
# conn = sqlite3.connect('db')
# # заливка таблицы df в  'db' базу
# df.to_sql('test_table', conn, index=False, if_exists='replace')
# # чтение запроса
# pd.read_sql(sql, conn)
# # функция select, для короткой записи запроса
# def select(sql):
#     return pd.read_sql(sql, conn)

In [4]:
# german_credit = pd.read_csv('/Users/maksa/Folder exp/german_credit_augmented.csv')
# german_credit['contract_dt'] = pd.to_datetime(german_credit['contract_dt'],format='%Y-%m-%d %H:%M:%S')
# # заливка таблицы german_credit в  'db' базу
# german_credit.to_sql('german_credit', conn, index=False, if_exists='replace')

In [5]:
# connect
conn = sqlite3.connect('db')
# функция select, для короткой записи запроса
def select(sql):
  return pd.read_sql(sql,conn)

In [6]:
sql = '''select * from german_credit limit 3'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200


In [8]:
# подсчет кол-ва строк в выдаче
sql = '''select count(*) from german_credit t 
where t.contract_dt between '2007-01-01' and '2007-12-31'
'''
select(sql)

,count(*)
0,573


In [9]:
# условие: case when; iif() 

sql = '''select t.credit_amount,
case when t.credit_amount > 1000 then 1 else 0 end as greater_1000_flag,

iif(t.credit_amount > 1000,1,0) as greater_1000_flag2

 from german_credit t'''
select(sql)

,credit_amount,greater_1000_flag,greater_1000_flag2
0,3074,1,1
1,1344,1,1
2,936,0,0
3,1393,1,1
4,776,0,0
...,...,...,...
995,2600,1,1
996,4455,1,1
997,6403,1,1
998,5003,1,1


In [10]:
# создание новой таблицы через запрос
cur = conn.cursor()

In [11]:
sql = '''
drop table if exists greater_1000_credit;  -- удаление таблицы на всякий случай

create table greater_1000_credit as        -- создание таблицы
select * from german_credit t
where t.credit_amount > 1000'''

cur.executescript(sql)  # запуск скрипта

In [12]:
sql = '''select * from greater_1000_credit t'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
3,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
4,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
...,...,...,...,...,...,...,...,...,...,...,...,...
879,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
880,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
881,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
882,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


In [13]:
jan = pd.DataFrame({'month':['jan','jan'],'revenue':[1,2]})
feb = pd.DataFrame({'month':['feb','feb'],'revenue':[1,2]})

In [14]:
jan.to_sql('jan', conn, index=False, if_exists='replace')
feb.to_sql('feb', conn, index=False, if_exists='replace')

In [15]:
# объединение(по вертикали) вывода запроса "union all"  и  "union" (удалит дубликаты)
sql = '''select * from jan t
union all
select * from feb t'''
select(sql)

,month,revenue
0,jan,1
1,jan,2
2,feb,1
3,feb,2


In [17]:
sql = '''select t.revenue from jan t
union
select t.revenue from feb t'''
select(sql)

,revenue
0,1
1,2


In [18]:
# первая буква слова
sql = '''select t.sex, substr(t.sex,1,1) from german_credit t'''
select(sql)

,sex,"substr(t.sex,1,1)"
0,male,m
1,male,m
2,male,m
3,female,f
4,male,m
...,...,...
995,male,m
996,male,m
997,male,m
998,female,f


#### Сводные таблицы

In [19]:
sql = '''
select 
t.sex,

count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by t.sex
 '''
select(sql)

,sex,cnt,credit_amount_avg
0,female,310,2877.774194
1,male,690,3448.040580


In [21]:
german_credit = pd.read_csv('/Users/maksa/Folder exp/german_credit_augmented.csv')
german_credit.groupby('sex')['credit_amount'].agg(['count','mean'])

,count,mean
sex,,
female,310,2877.774194
male,690,3448.040580


In [22]:
sql = '''select count(distinct t.housing), count(t.housing) from german_credit t'''
select(sql)

,count(distinct t.housing),count(t.housing)
0,3,1000


In [23]:
# пропуски
sql = '''select count(t.checking_account), count(0) from german_credit t'''
select(sql)

,count(t.checking_account),count(0)
0,606,1000


In [24]:
sql = '''
select 
t.checking_account,

count(*) as cnt,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by t.checking_account
 '''
select(sql)

,checking_account,cnt,credit_amount_avg
0,None,394,3133.101523
1,little,274,3175.218978
2,moderate,269,3827.561338
3,rich,63,2177.650794


In [26]:
# ! "is null" , а не  "==0"
sql = '''select 
sum(case when t.checking_account is null then 1 else 0 end) as is_null,

count(case when t.checking_account is null then 1 else null end) as is_null2

 from german_credit t'''
select(sql)

,is_null,is_null2
0,394,394


In [28]:
# coalesce(,) замена Nan на другое значение, или !!! coalesce(t.col1,t.col2,t.col3) выбираем первое не пустое значение !!!
sql = '''select
t.checking_account,

coalesce(t.checking_account,'no_info')

 from german_credit t'''
select(sql).head()

,checking_account,"coalesce(t.checking_account,'no_info')"
0,None,no_info
1,little,little
2,None,no_info
3,None,no_info
4,None,no_info


In [31]:
# поиск дубликатов
sql = '''select t.credit_amount, count(1) as cnt
from german_credit t
group by t.credit_amount
having count(1) > 1
'''
select(sql).head()

,credit_amount,cnt
0,433,2
1,609,2
2,701,2
3,709,2
4,717,2


In [32]:
# Аггрегация (количество кредитов в месяц)
sql = '''select date(t.contract_dt,'start of month') as month,  -- вытаскиваем месяц
count(1) as credit_cnt,
count(distinct t.client_id) as client_id_unique,
sum(t.credit_amount) as credit_amount_sum,
avg(t.credit_amount) as credit_amount_avg

 from german_credit t
 group by date(t.contract_dt,'start of month')  -- лучше целиком запрос, а не элиас
 order by date(t.contract_dt,'start of month')
 '''
select(sql)

,month,credit_cnt,client_id_unique,credit_amount_sum,credit_amount_avg
0,2007-05-01,81,81,207663,2563.740741
1,2007-06-01,74,74,239594,3237.756757
2,2007-07-01,71,71,224333,3159.619718
3,2007-08-01,57,57,178569,3132.789474
4,2007-09-01,58,58,186909,3222.568966
5,2007-10-01,70,70,188534,2693.342857
6,2007-11-01,87,87,300504,3454.068966
7,2007-12-01,77,77,273973,3558.090909
8,2008-01-01,93,93,288080,3097.634409
9,2008-02-01,55,55,211128,3838.690909


In [34]:
sql = '''select 

case 
when t.credit_amount < 1000 then '1. <1000'
when t.credit_amount < 2000 then '2. 1000-2000' 
when t.credit_amount < 3000 then '3. 2000-3000'
when t.credit_amount >= 3000 then '4. >= 3000'
else 'other' end as credit_amount_bin,

count(*) as credit_cnt                -- чтобы посчитать 

 from german_credit t
 group by credit_amount_bin
 '''
select(sql)

,credit_amount_bin,credit_cnt
0,1. <1000,116
1,2. 1000-2000,316
2,3. 2000-3000,188
3,4. >= 3000,380


In [36]:
# опять сводные таблицы - pivot
sql = '''select t.housing, 

count(case when t.sex = 'female' then 1 else null end) as female,  -- отдельно female
count(case when t.sex = 'male' then 1 else null end) as male,      -- отдельно male, за счёт "else null"

count(*) as cnt 

from german_credit t
group  by t.housing'''
select(sql)

,housing,female,male,cnt
0,free,19,89,108
1,own,196,517,713
2,rent,95,84,179


In [ ]:
# продолжаем ...

In [37]:
# выбираем все значения t.purpose - цели кредита
sql = '''select distinct t.purpose from german_credit t'''
purpose = list(select(sql)['purpose'].values)
purpose

['radio/TV',
 'car',
 'education',
 'furniture/equipment',
 'repairs',
 'business',
 'domestic appliances',
 'vacation/others']

In [38]:
# генерируем все строки запроса, чтобы не писать в ручную
for p in purpose:
  print(f"count(case when t.purpose = '{p}' then 1 else null end) as {p.lower().replace(' ','').replace('/','')},")

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,


In [39]:
sql = '''select t.housing, 

count(case when t.purpose = 'radio/TV' then 1 else null end) as radiotv,
count(case when t.purpose = 'car' then 1 else null end) as car,
count(case when t.purpose = 'education' then 1 else null end) as education,
count(case when t.purpose = 'furniture/equipment' then 1 else null end) as furnitureequipment,
count(case when t.purpose = 'repairs' then 1 else null end) as repairs,
count(case when t.purpose = 'business' then 1 else null end) as business,
count(case when t.purpose = 'domestic appliances' then 1 else null end) as domesticappliances,
count(case when t.purpose = 'vacation/others' then 1 else null end) as vacationothers,
count(1) as cnt 

from german_credit t
group  by t.housing'''
select(sql)

,housing,radiotv,car,education,furnitureequipment,repairs,business,domesticappliances,vacationothers,cnt
0,free,15,55,15,11,3,5,0,4,108
1,own,227,219,34,122,17,76,10,8,713
2,rent,38,63,10,48,2,16,2,0,179


In [41]:
# create table
t = pd.DataFrame({'purpose':['машина','машина','машина','на машину','на покупку машины',
                             'автомобиль','на возвращение 2007', 
                             'на свадьбу','свадьба','свадьба','свадьба','для свадьбы',
                             'недвижимость','на покупку недвижимости']})
t.to_sql('purpose', conn, index=False, if_exists='replace')

In [42]:
sql = '''select * from purpose t'''
select(sql)

,purpose
0,машина
1,машина
2,машина
3,на машину
4,на покупку машины
5,автомобиль
6,на возвращение 2007
7,на свадьбу
8,свадьба
9,свадьба


In [43]:
# Делим на категории. LIKE
# используем f'-string и подзапрос!!!

In [44]:
cat = '''select t.purpose,

case when t.purpose like '%свадьб%' then 'свадьба'                     -- LIKE
when t.purpose like '%машин%' or t.purpose like '%авто%' then 'машина'
when t.purpose like '%недвиж%' then 'недвижимость'

else 'другое' end as purpose_cat

 from purpose t'''

In [45]:
sql = f'''select            -- f'строка
t.purpose_cat,

count(1)

 from ({cat}) t             -- переменная и подзапрос одновременно !!!
 group by t.purpose_cat'''
select(sql)

,purpose_cat,count(1)
0,другое,1
1,машина,6
2,недвижимость,2
3,свадьба,5


#### Подзапросы

In [47]:
t = pd.DataFrame({'id':[1,1,2,2,3],
                  'name':['a','b','c','d','e']})
t.to_sql('dupl_test',conn,index=False,if_exists='replace')

In [48]:
sql = '''select * from dupl_test t'''
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [50]:
sql = '''select t.id from dupl_test t
group by t.id
having count(*) > 1'''
select(sql)

,id
0,1
1,2


In [52]:
 #  in Where
sql = '''select * from dupl_test t
where t.id in 
    (select t.id from dupl_test t
    group by t.id
    having count(1) > 1)'''
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [54]:
 #  in From
sql = '''
select * from 

(select t.id, count(1) as cnt 
from dupl_test t
group by t.id) t

where t.cnt > 1
'''
select(sql)

,id,cnt
0,1,2
1,2,2


#### CTE (With)

In [56]:
sql = '''
with id_cnt as (
select t.id,count(1) as cnt from dupl_test t
group by t.id)

select * from id_cnt t
where t.cnt > 1
'''
select(sql)

,id,cnt
0,1,2
1,2,2


In [59]:
# первый with as id_cnt, потом второй as id_cnt_2 который !!!использует первый, а потом select
sql = '''
with 
    id_cnt as 
    (select t.id,count(1) as cnt from dupl_test t
    group by t.id),

    id_cnt_2 as
    (select * from id_cnt t
    where t.cnt > 1)

select * 
from id_cnt_2 t
where t.id = 1

'''
select(sql)

,id,cnt
0,1,2


#### Джойны

In [60]:
users = pd.DataFrame({'id':[1,2,3],'name':['gleb','jon snow','tyrion']})
items = pd.DataFrame({'user_id':[1,3,3],'item_name':['hleb','gold','wine'],'value':[5,100,20]})

In [61]:
users.to_sql('users',conn,index=False,if_exists='replace')
items.to_sql('items',conn,index=False,if_exists='replace')

In [63]:
# LEFT
sql = '''select t.*, i.item_name from users t
left join items i on t.id = i.user_id
'''
select(sql)

,id,name,item_name
0,1,gleb,hleb
1,2,jon snow,None
2,3,tyrion,gold
3,3,tyrion,wine


In [65]:
# INNER - пропала строка с snow т.к. его нет во второй таблице
sql = '''select t.*, i.item_name from users t
inner join items i on t.id = i.user_id
'''
select(sql)

,id,name,item_name
0,1,gleb,hleb
1,3,tyrion,gold
2,3,tyrion,wine


In [66]:
# INNER по умолчанию? - пропала строка с snow т.к. его нет во второй таблице
sql = '''select t.*, i.item_name from users t
join items i on t.id = i.user_id
'''
select(sql)

,id,name,item_name
0,1,gleb,hleb
1,3,tyrion,gold
2,3,tyrion,wine


In [67]:
sql = '''select t.*, i.item_name, i.value, i.user_id from users t
left join items i on t.id = i.user_id
'''
select(sql)

,id,name,item_name,value,user_id
0,1,gleb,hleb,5.0,1.0
1,2,jon snow,None,NaN,NaN
2,3,tyrion,gold,100.0,3.0
3,3,tyrion,wine,20.0,3.0


! Агрегируй перед тем как джойнить

In [68]:
#  Не так !!!
sql = '''select t.id, t.name, 
count(i.item_name) as item_cnt,
coalesce(sum(i.value),0) as value_sum

from users t
left join items i on t.id = i.user_id
group by t.id, t.name
'''
select(sql)

,id,name,item_cnt,value_sum
0,1,gleb,1,5
1,2,jon snow,0,0
2,3,tyrion,2,120


In [70]:
# Вот так !!!
sql = '''
with items_agg as (
-- аггрегируем в "новую" таблицу
select t.user_id, count(t.item_name) as item_cnt,
sum(value) as value_sum from items t
group by t.user_id)

select t.id, t.name,
coalesce(i.item_cnt,0) as item_cnt,
coalesce(i.value_sum,0) as value_sum

from users t

left join items_agg i on t.id = i.user_id

'''
select(sql)

,id,name,item_cnt,value_sum
0,1,gleb,1,5
1,2,jon snow,0,0
2,3,tyrion,2,120


In [71]:
# задаём таблицы через SQL
sql = '''select 1 as user_id
union all
select 2 as user_id
union all
select 3 as user_id'''
select(sql)

,user_id
0,1
1,2
2,3


In [72]:
sql = '''select date('2021-03-01') as month
union all
select date('2021-04-01') as month
'''
select(sql)

,month
0,2021-03-01
1,2021-04-01


In [73]:
# "Cross"join
sql = '''
with users as (
select 1 as user_id
union all
select 2 as user_id
union all
select 3 as user_id),

month as

(select date('2021-03-01') as month
union all
select date('2021-04-01') as month)


select * from users t
join month m on 1=1     -- каждую строку к каждой строке !!!

'''
select(sql)

,user_id,month
0,1,2021-03-01
1,2,2021-03-01
2,3,2021-03-01
3,1,2021-04-01
4,2,2021-04-01
5,3,2021-04-01


Ежемесячный отчет:

In [74]:
transactions = pd.read_csv('/Users/maksa/Folder exp/german_credit_augmented_transactions.csv')

In [76]:
transactions.to_sql('client_transactions',conn,index=False,if_exists='replace')
sql = '''select * from client_transactions t limit 5'''
select(sql)

,dt,client_id,amount
0,2008-04-06 11:54:47,950,161.38
1,2007-07-28 00:00:19,418,35.34
2,2008-03-14 20:43:54,131,146.50
3,2007-12-18 13:03:24,353,119.21
4,2007-11-09 05:18:30,849,105.24


In [77]:
min = '''select date(min(t.dt),'start of month') from client_transactions t'''
max = '''select date(max(t.dt),'start of month') from client_transactions t'''

In [79]:
# задаем список нужных дат(чтобы ничего не упустить)
sql = f'''WITH dates(month) AS (
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
)
SELECT t.month FROM dates t'''
select(sql)

,month
0,2007-05-01
1,2007-06-01
2,2007-07-01
3,2007-08-01
4,2007-09-01
5,2007-10-01
6,2007-11-01
7,2007-12-01
8,2008-01-01
9,2008-02-01


In [80]:
sql = f'''WITH 
dates(month) AS (
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
),

trans_month as
(select 
date(t.dt, 'start of month') as month,
count(1) as transaction_cnt,
sum(t.amount) as amount_sum

from client_transactions t
group by 1                 --  по первому столбцу(t.dt)
order by 1)


SELECT t.month,
coalesce(tm.transaction_cnt,0) as transaction_cnt,
coalesce(tm.amount_sum,0) as amount_sum

FROM dates t
left join trans_month tm on t.month = tm.month
order by t.month'''
select(sql)

,month,transaction_cnt,amount_sum
0,2007-05-01,338,450912.77
1,2007-06-01,379,551664.83
2,2007-07-01,304,494134.50
3,2007-08-01,255,426903.23
4,2007-09-01,0,0.00
5,2007-10-01,332,634846.49
6,2007-11-01,389,500420.98
7,2007-12-01,364,561449.89
8,2008-01-01,413,630137.22
9,2008-02-01,228,337043.47


а могли пропустить 2007-09-01	0	0.00, если бы не посчитали месяца отдельно и не сджойнили

In [82]:
# пример с пропуском
sql = '''select 
date(t.dt, 'start of month') as month,
count(*) as transaction_cnt,
sum(t.amount) as amount_sum

from client_transactions t
group by 1                 --  по первому столбцу(t.dt)
order by 1'''
select(sql)

,month,transaction_cnt,amount_sum
0,2007-05-01,338,450912.77
1,2007-06-01,379,551664.83
2,2007-07-01,304,494134.50
3,2007-08-01,255,426903.23
4,2007-10-01,332,634846.49
5,2007-11-01,389,500420.98
6,2007-12-01,364,561449.89
7,2008-01-01,413,630137.22
8,2008-02-01,228,337043.47
9,2008-03-01,309,425599.09


Ежемесячный отчет на пользователя:

In [87]:
sql = '''select 
date(t.dt, 'start of month') as month,
t.client_id,
count(*) as transaction_cnt,
sum(t.amount) as amount_sum

 from client_transactions t
group by 1,2
order by 1'''
select(sql)

,month,client_id,transaction_cnt,amount_sum
0,2007-05-01,101,1,149.67
1,2007-05-01,107,2,217.50
2,2007-05-01,110,1,1081.32
3,2007-05-01,111,1,139.97
4,2007-05-01,113,1,1051.29
...,...,...,...,...
3551,2008-06-01,983,1,10050.37
3552,2008-06-01,987,1,192.48
3553,2008-06-01,992,1,-610.21
3554,2008-06-01,996,1,121.28


In [88]:
sql = f'''WITH dates(month) AS (                      -- 1 with
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
),

clients as (select distinct t.client_id from german_credit t),    -- 2 with

clients_month as                                      -- 3 with = 1w + 2w
(SELECT t.month, c.client_id 
FROM dates t
join clients c on 1=1),                               -- crossjoin

trans_month as                                        -- 4 with = 1w + table client_transactions
(select 
date(t.dt, 'start of month') as month,
t.client_id,
count(1) as transaction_cnt,
sum(t.amount) as amount_sum

 from client_transactions t
group by 1,2
order by 1),

client_trans_month as                                  -- 5 with = 3w + 4w
(select t.*,
tm.transaction_cnt,
tm.amount_sum,
1 as user,                                                       -- считаем пользователей
case when tm.transaction_cnt > 0 then 1 else 0 end as active     -- (1-есть, 0-небыло)

 from clients_month t
left join trans_month tm on t.client_id = tm.client_id
and t.month = tm.month
)

select t.month, sum(t.user) as user_cnt,                -- from 5w = result
sum(t.amount_sum) as amount_sum , 
sum(t.active) as active_cnt 
from client_trans_month t
group by t.month
'''
select(sql)

,month,user_cnt,amount_sum,active_cnt
0,2007-05-01,1000,450912.77,288
1,2007-06-01,1000,551664.83,297
2,2007-07-01,1000,494134.50,259
3,2007-08-01,1000,426903.23,222
4,2007-09-01,1000,NaN,0
5,2007-10-01,1000,634846.49,283
6,2007-11-01,1000,500420.98,323
7,2007-12-01,1000,561449.89,287
8,2008-01-01,1000,630137.22,325
9,2008-02-01,1000,337043.47,204


Джойн таблицы самой на себя (нарастающий итог)

In [89]:
t = pd.DataFrame({'dt':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03'],format='%Y-%m-%d'),
                  'revenue':[1,2,3]})

In [93]:
t

,dt,revenue
0,2021-04-01,1
1,2021-04-02,2
2,2021-04-03,3


In [91]:
t.to_sql('revenue',conn,index=False,if_exists='replace')

In [95]:
sql = '''select 
t.dt, r.dt
from revenue t
join revenue r on r.dt <= t.dt   -- нарастающий итог
'''
select(sql)

,dt,dt
0,2021-04-01 00:00:00,2021-04-01 00:00:00
1,2021-04-02 00:00:00,2021-04-01 00:00:00
2,2021-04-02 00:00:00,2021-04-02 00:00:00
3,2021-04-03 00:00:00,2021-04-01 00:00:00
4,2021-04-03 00:00:00,2021-04-02 00:00:00
5,2021-04-03 00:00:00,2021-04-03 00:00:00


In [92]:
sql = '''select 
t.dt, t.revenue, sum(r.revenue) as cumsum 
from revenue t
join revenue r on r.dt <= t.dt   -- нарастающий итог
group by t.dt, t.revenue'''
select(sql)

,dt,revenue,cumsum
0,2021-04-01 00:00:00,1,1
1,2021-04-02 00:00:00,2,3
2,2021-04-03 00:00:00,3,6


#### Оконные функции

In [96]:
# тот же нарастающий итог, но без джойна
sql = '''select t.*,
sum(t.revenue) over (order by t.dt) as cum_sum
from revenue t'''
select(sql)

,dt,revenue,cum_sum
0,2021-04-01 00:00:00,1,1
1,2021-04-02 00:00:00,2,3
2,2021-04-03 00:00:00,3,6


In [97]:
t = pd.DataFrame({'user_id':[1,1,1,2,2,2],
                  'dt':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03',
                                       '2021-04-01','2021-04-02','2021-04-03'],
                                        format='%Y-%m-%d'),
                  'revenue':[1,2,3,2,3,4]})

In [98]:
t

,user_id,dt,revenue
0,1,2021-04-01,1
1,1,2021-04-02,2
2,1,2021-04-03,3
3,2,2021-04-01,2
4,2,2021-04-02,3
5,2,2021-04-03,4


In [99]:
t.to_sql('revenue', conn, index=False, if_exists='replace')

In [ ]:
# окно задаёт "partition by"(разделять по, группировать по), синтаксис: "функция over (partition by t.x order by t.y) as xy"

In [100]:
sql = '''select t.*,
sum(t.revenue) over (partition by t.user_id order by t.dt) as cum_sum
from revenue t'''
select(sql)

,user_id,dt,revenue,cum_sum
0,1,2021-04-01 00:00:00,1,1
1,1,2021-04-02 00:00:00,2,3
2,1,2021-04-03 00:00:00,3,6
3,2,2021-04-01 00:00:00,2,2
4,2,2021-04-02 00:00:00,3,5
5,2,2021-04-03 00:00:00,4,9


In [101]:
# rank() - ранжирует по окну, ранги могут совпадать(даже у одного юзера, если например даты одинаковые)
sql = '''select t.*,

rank() over (partition by t.user_id order by t.dt desc) as rnk

 from revenue t'''
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,3,1
1,1,2021-04-02 00:00:00,2,2
2,1,2021-04-01 00:00:00,1,3
3,2,2021-04-03 00:00:00,4,1
4,2,2021-04-02 00:00:00,3,2
5,2,2021-04-01 00:00:00,2,3


In [102]:
sql = '''
with dt_rank as (
select t.*,
rank() over (partition by t.user_id order by t.dt desc) as rnk
from revenue t)
 
select * from dt_rank t
where t.rnk = 1                     -- выбираем наибольшую дату
'''
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,3,1
1,2,2021-04-03 00:00:00,4,1


In [103]:
# row_number() - нумерует строки по окну, номера не могут совпадать
sql = '''select t.*,

row_number() over (partition by t.user_id order by t.dt desc) as rnk

 from revenue t'''
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,3,1
1,1,2021-04-02 00:00:00,2,2
2,1,2021-04-01 00:00:00,1,3
3,2,2021-04-03 00:00:00,4,1
4,2,2021-04-02 00:00:00,3,2
5,2,2021-04-01 00:00:00,2,3


In [104]:
sql = '''
with dt_rank as (
select t.*,
row_number() over (partition by t.user_id order by t.dt desc) as rnk
from revenue t)
 
select * from dt_rank t
where t.rnk = 1                     -- выбираем наибольшую дату
'''
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,3,1
1,2,2021-04-03 00:00:00,4,1


In [ ]:
# тоже самое можно сделать через max(date) и join, но ///

In [105]:
sql = '''
with last_dt as (
select t.user_id, max(t.dt) as max_dt
from revenue t
group by t.user_id)

select t.* from revenue t
join last_dt ld on t.user_id = ld.user_id and t.dt = ld.max_dt
'''
select(sql)

,user_id,dt,revenue
0,1,2021-04-03 00:00:00,3
1,2,2021-04-03 00:00:00,4


Топ 3 зарплаты в отделе:

In [106]:
t = pd.DataFrame({'dep':['a','a','a','a','a',      # департамент
                         'b','b','b','b','b'],
                  'emp':['aa','bb','cc','dd','ee', # должность
                         'aa','bb','cc','dd','ee'],
                  'sal':[5,5,3,2,1,                # зарплаты
                         5,4,3,2,1]})
t

,dep,emp,sal
0,a,aa,5
1,a,bb,5
2,a,cc,3
3,a,dd,2
4,a,ee,1
5,b,aa,5
6,b,bb,4
7,b,cc,3
8,b,dd,2
9,b,ee,1


In [108]:
t.to_sql('salary',conn,index=False,if_exists='replace')

In [109]:
# dense_rank() - ранжирует подряд(1-1-2-3, а не 1-1-3-4)
sql = '''select t.*,
rank() over (partition by t.dep order by t.sal desc) as rnk_rank,
dense_rank() over (partition by t.dep order by t.sal desc) as rnk

from salary t'''
select(sql)

,dep,emp,sal,rnk_rank,rnk
0,a,aa,5,1,1
1,a,bb,5,1,1
2,a,cc,3,3,2
3,a,dd,2,4,3
4,a,ee,1,5,4
5,b,aa,5,1,1
6,b,bb,4,2,2
7,b,cc,3,3,3
8,b,dd,2,4,4
9,b,ee,1,5,5


In [110]:
sql = '''
with salary_rnk as (
select t.*,
dense_rank() over (partition by t.dep order by t.sal desc) as rnk
from salary t)
 
select * from salary_rnk t
where t.rnk <= 3
'''
select(sql)

,dep,emp,sal,rnk
0,a,aa,5,1
1,a,bb,5,1
2,a,cc,3,2
3,a,dd,2,3
4,b,aa,5,1
5,b,bb,4,2
6,b,cc,3,3


Клиентские сессии (одна сессия - заходы с разницей меньше 1 часа)

In [111]:
user1 = pd.DataFrame({'user_id':[1,1,1,1,1],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-01 07:35',
                                       '2021-04-01 08:20','2021-04-01 12:31',
                                       '2021-04-03 07:31'],format='%Y-%m-%d %H:%M')})
user2 = pd.DataFrame({'user_id':[2,2,2,2],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-01 07:35',
                                       '2021-04-01 08:20','2021-04-01 9:10'
                                       ],format='%Y-%m-%d %H:%M')})
user3 = pd.DataFrame({'user_id':[3,3,3],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-02 07:35',
                                       '2021-04-03 08:20'
                                       ],format='%Y-%m-%d %H:%M')})

In [112]:
t = pd.concat([user1,user2,user3])
t

,user_id,dt
0,1,2021-04-01 07:31:00
1,1,2021-04-01 07:35:00
2,1,2021-04-01 08:20:00
3,1,2021-04-01 12:31:00
4,1,2021-04-03 07:31:00
0,2,2021-04-01 07:31:00
1,2,2021-04-01 07:35:00
2,2,2021-04-01 08:20:00
3,2,2021-04-01 09:10:00
0,3,2021-04-01 07:31:00


In [113]:
t.to_sql('client_log', conn, index=False, if_exists='replace')

In [116]:
# lag() - предыдущее
sql = '''select t.*,
lag(t.dt) over (partition by t.user_id order by t.dt) as prev_dt,                -- предыдущее время
--  разница между настоящим и пред.временем в секундах
round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) as dt_diff,

case when round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) >= 3600
then 1 else 0 end as new_session    -- cчитаем новые сессии (прошло больше часа)


from client_log t'''
select(sql)

,user_id,dt,prev_dt,dt_diff,new_session
0,1,2021-04-01 07:31:00,None,NaN,0
1,1,2021-04-01 07:35:00,2021-04-01 07:31:00,240.0,0
2,1,2021-04-01 08:20:00,2021-04-01 07:35:00,2700.0,0
3,1,2021-04-01 12:31:00,2021-04-01 08:20:00,15060.0,1
4,1,2021-04-03 07:31:00,2021-04-01 12:31:00,154800.0,1
5,2,2021-04-01 07:31:00,None,NaN,0
6,2,2021-04-01 07:35:00,2021-04-01 07:31:00,240.0,0
7,2,2021-04-01 08:20:00,2021-04-01 07:35:00,2700.0,0
8,2,2021-04-01 09:10:00,2021-04-01 08:20:00,3000.0,0
9,3,2021-04-01 07:31:00,None,NaN,0


In [123]:
sql = '''
with
new_session as (
select t.*, lag(t.dt) over (partition by t.user_id order by t.dt) as prev_dt,
    --  разница между настоящим и пред.временем в секундах
round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) as dt_diff,
case when round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) >= 3600
then 1 else 0 end as new_session    -- cчитаем новые сессии (прошло больше часа)
from client_log t)

select t.*,
    -- (окно) нарастающий итог по new_session: результат - номер сессии по порядку
sum(t.new_session) over (partition by t.user_id order by t.dt) as session_id
from new_session t
'''
select(sql)

,user_id,dt,prev_dt,dt_diff,new_session,session_id
0,1,2021-04-01 07:31:00,None,NaN,0,0
1,1,2021-04-01 07:35:00,2021-04-01 07:31:00,240.0,0,0
2,1,2021-04-01 08:20:00,2021-04-01 07:35:00,2700.0,0,0
3,1,2021-04-01 12:31:00,2021-04-01 08:20:00,15060.0,1,1
4,1,2021-04-03 07:31:00,2021-04-01 12:31:00,154800.0,1,2
5,2,2021-04-01 07:31:00,None,NaN,0,0
6,2,2021-04-01 07:35:00,2021-04-01 07:31:00,240.0,0,0
7,2,2021-04-01 08:20:00,2021-04-01 07:35:00,2700.0,0,0
8,2,2021-04-01 09:10:00,2021-04-01 08:20:00,3000.0,0,0
9,3,2021-04-01 07:31:00,None,NaN,0,0


In [128]:
# количество активностей за сессию у клиента
sql = '''
with
new_session as (
select t.*, lag(t.dt) over (partition by t.user_id order by t.dt) as prev_dt,
    --  разница между настоящим и пред.временем в секундах
round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) as dt_diff,
case when round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) >= 3600
then 1 else 0 end as new_session    -- cчитаем новые сессии (прошло больше часа)
from client_log t),

client_sessions as (
select t.*,
    -- (окно)нарастающий итог по new_session: результат - номер сессии по порядку
sum(t.new_session) over (partition by t.user_id order by t.dt) as session_id
from new_session t)

select t.user_id, t.session_id, count(*) as action_cnt  -- или count(session_id)
from client_sessions t
group by t.user_id, t.session_id
'''
select(sql)

,user_id,session_id,action_cnt
0,1,0,3
1,1,1,1
2,1,2,1
3,2,0,4
4,3,0,1
5,3,1,1
6,3,2,1


In [129]:
# количество сессий которые есть в базе
sql = '''
with
new_session as (
select t.*, lag(t.dt) over (partition by t.user_id order by t.dt) as prev_dt,
    --  разница между настоящим и пред.временем в секундах
round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) as dt_diff,
case when round((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) >= 3600
then 1 else 0 end as new_session    -- cчитаем новые сессии (прошло больше часа)
from client_log t),

client_sessions as (
select t.*,
    -- (окно)нарастающий итог по new_session: результат - номер сессии по порядку
sum(t.new_session) over (partition by t.user_id order by t.dt) as session_id
from new_session t),

client_sessions_agg as (
select t.user_id, t.session_id, count(*) as action_cnt 
from client_sessions t
group by t.user_id, t.session_id)

select count(*) 
from client_sessions_agg t
'''
select(sql)

,count(*)
0,7


Скользящее среднее

In [133]:
t = pd.DataFrame({'user_id':[1,1,1,1,1,1,
                             2,2,2,2,2],
                  'dt':[1,2,3,4,5,6,       # дата (как бы)
                        1,2,3,4,5],
                  'revenue':[1,2,3,4,5,6,
                             3,4,5,6,7]})
t.to_sql('revenue', conn, index=False, if_exists='replace')
t

,user_id,dt,revenue
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
5,1,6,6
6,2,1,3
7,2,2,4
8,2,3,5
9,2,4,6


In [134]:
# moving_avg 3 days
sql = '''
select t.*, 
       avg(t.revenue) over (partition by t.user_id 
                            order by t.dt 
                            rows between 2 preceding and current row)
                            as moving_avg
from revenue t'''
select(sql)

,user_id,dt,revenue,moving_avg
0,1,1,1,1.0
1,1,2,2,1.5
2,1,3,3,2.0
3,1,4,4,3.0
4,1,5,5,4.0
5,1,6,6,5.0
6,2,1,3,3.0
7,2,2,4,3.5
8,2,3,5,4.0
9,2,4,6,5.0
